In [2]:
import pandas as pd
import numpy as np
import random

In [3]:
df = pd.read_csv('movies_with_test.csv')

In [4]:
df.shape

(199, 2)

In [5]:
def create_shingles(text, min_length=1, max_length=None):
  tags = [tag.strip() for tag in text.split(',')]  # Tách các tag riêng biệt

  all_shingles = set()
  for tag in tags:
      words = tag.split()  # Tách tag thành các từ
      if max_length is None:
          max_length = max(len(word) for word in words)
      for word in words:
          for i in range(min_length, min(max_length, len(word)) + 1):
              all_shingles.add(word[:i])
  return all_shingles

df['shingles'] = df['genres'].astype(str).apply(create_shingles)

In [6]:
df['shingles']

0                      {Com, Comedy, C, Co, Come, Comed}
1                              {D, Dr, Dra, Dram, Drama}
2      {Com, Adult, C, Adu, Adul, A, Co, Come, Comed,...
3      {Sh, Family, Animatio, Famil, Short, Anim, Sho...
4                              {D, Dr, Dra, Dram, Drama}
                             ...                        
194                            {D, Dr, Dra, Dram, Drama}
195                            {D, Dr, Dra, Dram, Drama}
196    {Com, Advent, Comedy, Adven, Animatio, Adventu...
197                    {Com, Comedy, C, Co, Come, Comed}
198    {Com, Advent, Comedy, Adven, Animatio, Adventu...
Name: shingles, Length: 199, dtype: object

In [7]:
def create_hash_functions(num_hash_functions, max_val):
    hash_functions = []
    for _ in range(num_hash_functions):
        a = random.randint(1, max_val)
        b = random.randint(0, max_val)
        hash_functions.append(lambda x, a=a, b=b: (a * x + b) % max_val)
    return hash_functions

def minhash_signature(shingles, hash_functions):
    signature = [float('inf')] * len(hash_functions) 
    for shingle in shingles:
        shingle_hash = hash(shingle)
        for i, hash_func in enumerate(hash_functions):
            signature[i] = min(signature[i], hash_func(shingle_hash))
    return signature

all_shingles = set()
for shingles_list in df['shingles']:
    all_shingles.update(shingles_list)
# Create 10 hash functions
hash_functions = create_hash_functions(10, len(all_shingles))

# Apply minhash_signature to each row in the `shingles` column
df['minhash_signature'] = df['shingles'].apply(lambda x: minhash_signature(x, hash_functions))


In [8]:
all_shingles

{'A',
 'Ac',
 'Act',
 'Acti',
 'Actio',
 'Action',
 'Ad',
 'Adu',
 'Adul',
 'Adult',
 'Adv',
 'Adve',
 'Adven',
 'Advent',
 'Adventu',
 'Adventur',
 'Adventure',
 'An',
 'Ani',
 'Anim',
 'Anima',
 'Animat',
 'Animati',
 'Animatio',
 'Animation',
 'B',
 'Bi',
 'Bio',
 'Biog',
 'Biogr',
 'Biogra',
 'Biograp',
 'Biograph',
 'Biography',
 'C',
 'Co',
 'Com',
 'Come',
 'Comed',
 'Comedy',
 'Cr',
 'Cri',
 'Crim',
 'Crime',
 'D',
 'Do',
 'Doc',
 'Docu',
 'Docum',
 'Docume',
 'Documen',
 'Document',
 'Documenta',
 'Documentar',
 'Documentary',
 'Dr',
 'Dra',
 'Dram',
 'Drama',
 'F',
 'Fa',
 'Fam',
 'Fami',
 'Famil',
 'Family',
 'Fan',
 'Fant',
 'Fanta',
 'Fantas',
 'Fantasy',
 'G',
 'Ga',
 'Gam',
 'Game',
 'Game-',
 'Game-S',
 'Game-Sh',
 'Game-Sho',
 'Game-Show',
 'H',
 'Hi',
 'His',
 'Hist',
 'Histo',
 'Histor',
 'History',
 'Ho',
 'Hor',
 'Horr',
 'Horro',
 'Horror',
 'M',
 'Mu',
 'Mus',
 'Musi',
 'Music',
 'Musica',
 'Musical',
 'My',
 'Mys',
 'Myst',
 'Myste',
 'Myster',
 'N',
 'Ne',
 'Ne

In [9]:
class LSH:
    def __init__(self, num_bands, num_rows_per_band):
        self.num_bands = num_bands
        self.num_rows_per_band = num_rows_per_band
        self.buckets = {}  # Lưu trữ các nhóm (bucket)


    def hash_function(self, signature, band_index):
        """
        Hàm băm đơn giản để ánh xạ một phần của chữ ký MinHash vào một nhóm.
        """
        start_row = band_index * self.num_rows_per_band
        end_row = start_row + self.num_rows_per_band
        band_signature = tuple(signature[start_row:end_row])
        return hash(band_signature)

    def insert(self, key, signature):
        """
        Chèn một mục (key) và chữ ký MinHash của nó vào các nhóm LSH.
        """
        for band_index in range(self.num_bands):
            bucket_id = self.hash_function(signature, band_index)
            if bucket_id not in self.buckets:
                self.buckets[bucket_id] = []
            self.buckets[bucket_id].append(key)

    def movie_similar(self, query_signature):
        """
        Truy vấn các mục có khả năng tương tự dựa trên chữ ký MinHash của truy vấn.
        """
        candidates = set()
        for band_index in range(self.num_bands):
            bucket_id = self.hash_function(query_signature, band_index)
            if bucket_id in self.buckets:
                candidates.update(self.buckets[bucket_id])
        return candidates
    def movie_query(self, tag_query):
        """
        Truy vấn các bộ phim có khả năng tương tự dựa trên tag truy vấn.
        """

        # Tạo tập hợp shingles từ tag truy vấn
        query_shingles = create_shingles(tag_query)

        # Chuyển đổi shingles thành MinHash signature (sử dụng hash_functions của LSH)
        query_signature = minhash_signature(query_shingles, hash_functions)    

        # Tìm các bộ phim ứng viên bằng LSH
        candidate_movie_ids = self.movie_similar(query_signature)

        return candidate_movie_ids

In [10]:
def jaccard_similarity(set1, set2):
    """
    Tính toán độ tương đồng Jaccard giữa hai tập hợp
    """
    intersection = len(set1.intersection(set2))
    union = len(set1.union(set2))
    return intersection / union

In [11]:
lsh = LSH(num_bands=5, num_rows_per_band=2)  # Chia chữ ký MinHash thành 5 bands, mỗi band có 2 hàng

# Chèn các bộ phim vào LSH
for index, row in df.iterrows():
    lsh.insert(row['originalTitle'], row['minhash_signature'])

# Truy vấn phim tương tự
query_movie = 'The Millionaire'
query_signature = df[df['originalTitle'] == query_movie]['minhash_signature'].iloc[0]
candidate_movies = lsh.movie_similar(query_signature)
print(candidate_movies)


{'Two Different Worlds', 'Her Boy Friend', 'Macao', 'The Utilizer', 'Hells Angels on Wheels', 'Kurvi', 'Episode #5.3', 'The Rats', 'Marzenia klauna', 'Orhideje prijatelja', 'Sinfonia Amazônica', 'Another Shell Game', 'Chapter 15', 'Antesala de la silla eléctrica', 'Tobunda ôzora he', "Mime's Eye", 'Episode #33.2', 'Niji no ike no jigoku janguru', 'Yoga for Little Brothers', 'Echt falsch, Teil 2', 'One Piece: Oounabara ni hirake! Dekkai dekkai chichi no yume!', 'The Leafblower', 'Horses Drawing in Seine', 'Super Dave: Daredevil for Hire', 'Naturaleza rota', 'Violent Delights', 'Roswell: The Unheard Broadcast', 'Destiny Love', 'Le deuil sied à Brigitte', 'Conscience', '100 eyes see their destiny', "Edgar's Feast Day", 'Seibutsu (Still:Life)', 'Aftermath', 'Maigret and the Night Club Dancer', 'The Millionaire', 'Façade', 'Episode dated 13 May 1993', 'Western Chivalry', 'The Pythoness', 'Navajo Indian Foot Race', 'Viernes 5 a.m.', 'Banchikwang', 'Episode #3.42', 'The Great Albert', "Ain't 

In [12]:
similarities = []
for candidate_movie in candidate_movies:
    candidate_signature = df[df['originalTitle'] == candidate_movie]['minhash_signature'].iloc[0]
    similarity = jaccard_similarity(set(query_signature), set(candidate_signature)) / len(hash_functions)
    similarities.append((candidate_movie, similarity))

threshold = 0.1
filtered_movies = [movie for movie, sim in similarities if sim >= threshold]
for movie, similarity in similarities:
    if movie != query_movie:  # Loại trừ chính bộ phim truy vấn
        print(f"Movie: {movie}, Jaccard Similarity: {similarity}")

Movie: Two Different Worlds, Jaccard Similarity: 0.03076923076923077
Movie: Her Boy Friend, Jaccard Similarity: 0.045454545454545456
Movie: Macao, Jaccard Similarity: 0.05454545454545454
Movie: The Utilizer, Jaccard Similarity: 0.023076923076923078
Movie: Hells Angels on Wheels, Jaccard Similarity: 0.04
Movie: Kurvi, Jaccard Similarity: 0.05
Movie: Episode #5.3, Jaccard Similarity: 0.05454545454545454
Movie: The Rats, Jaccard Similarity: 0.045454545454545456
Movie: Marzenia klauna, Jaccard Similarity: 0.05
Movie: Orhideje prijatelja, Jaccard Similarity: 0.04166666666666667
Movie: Sinfonia Amazônica, Jaccard Similarity: 0.05
Movie: Another Shell Game, Jaccard Similarity: 0.03333333333333333
Movie: Chapter 15, Jaccard Similarity: 0.025
Movie: Antesala de la silla eléctrica, Jaccard Similarity: 0.03076923076923077
Movie: Tobunda ôzora he, Jaccard Similarity: 0.03636363636363636
Movie: Mime's Eye, Jaccard Similarity: 0.04166666666666667
Movie: Episode #33.2, Jaccard Similarity: 0.05
Movie:

In [13]:
tag_query = "Ad"
query_result = lsh.movie_query(tag_query)
print(query_result)

{'World Sex Tour 22: Jamaica', 'Tit Teazer Files 1', 'Der Hausmeister', 'Sinfonia Amazônica', 'The Pythoness', 'Swedish Erotica 9', 'Meat My Ass 3', 'Super Dave: Daredevil for Hire', 'Denim', 'White Trash Whore 26', 'A Fox in a Fix', 'Chapter 15', 'Sinderella Live'}
